In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
from torchviz import make_dot

In [3]:
class Logistic:
    def __init__(self, X_train, Y_train, epoch=100, learning_rate=0.01):
        self.X_train = X_train
        self.Y_train = Y_train
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.Y_train = self.one_hot_encode_with_smoothing(Y_train)
        self.num_classes = self.Y_train.shape[1]
        self.num_features = self.X_train.shape[1]
        self.Weights = np.zeros((self.num_features, self.num_classes))
        self.Bias = np.zeros((1, self.num_classes))
        self.Weights_array = []
        self.Bias_array = []
        self.Weights_array.append(self.Weights)
        self.Bias_array.append(self.Bias)
        self.loss_array = []
        self.accuracy_array = []
        
    def one_hot_encode_with_smoothing(self, Y):
        one_hot = np.zeros((Y.size, Y.max() + 1))
        one_hot[np.arange(Y.size), Y] = 1
        smoothing_parameter = 0.1
        one_hot = one_hot * (1 - smoothing_parameter) + smoothing_parameter / one_hot.shape[1]
        return one_hot

    # ADD BOTH STOCHASTIC AND MINI BATCH GRADIENT DESCENT
    def train(self):
        for i in range(self.epoch):
            # print(f"Epoch {i+1} begins, value of Weights => {self.Weights}, value of Bias => {self.Bias}")
            activation = self.activation()
            # print(f"Activation => {activation}")
            grad_wrt_W = -np.matmul(self.X_train.T, (self.Y_train - activation))
            grad_wrt_B = -np.sum(self.Y_train - activation, axis=0, keepdims=True)

            self.Weights -= self.learning_rate * grad_wrt_W
            self.Bias -= self.learning_rate * grad_wrt_B

            self.Weights_array.append(self.Weights)
            self.Bias_array.append(self.Bias)

            # print(f"Epoch {i+1} ends, value of Weights => {self.Weights}, value of Bias => {self.Bias}")

            loss = self.loss(activation)
            self.loss_array.append(loss)
            accuracy = self.accuracy(activation)
            self.accuracy_array.append(accuracy)
            print(f"Epoch : {i+1} || Loss => {loss:.4f} || Accuracy => {accuracy:.4f}\n----------------------")

    def train_stochastic(self):
        for i in range(self.epoch):
            for j in range(self.X_train.shape[0]):
                activation = self.activation_stochastic(self.X_train[j].reshape(1, -1))
                grad_wrt_W = -np.matmul(self.X_train[j].reshape(-1, 1), (self.Y_train[j] - activation))
                grad_wrt_B = -np.sum(self.Y_train[j] - activation, axis=0, keepdims=True)

                self.Weights -= self.learning_rate * grad_wrt_W
                self.Bias -= self.learning_rate * grad_wrt_B

            self.Weights_array.append(self.Weights)
            self.Bias_array.append(self.Bias)

            activation = self.activation()
            loss = self.loss(activation)
            self.loss_array.append(loss)
            accuracy = self.accuracy(activation)
            self.accuracy_array.append(accuracy)
            print(f"Epoch : {i+1} || Loss => {loss:.4f} || Accuracy => {accuracy:.4f}\n----------------------")

    def activation_stochastic(self, X):
        linear_model = np.dot(X, self.Weights) + self.Bias
        return self.softmax(linear_model)

    def activation(self):
        linear_model = np.dot(self.X_train, self.Weights) + self.Bias
        return self.softmax(linear_model)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def loss(self, activation):
        n = self.Y_train.shape[0]
        m = self.Y_train.shape[1]
        total = 0
        for i in range(n):
            for j in range(m):
                total += self.Y_train[i, j] * np.log(activation[i, j])
        return -total / n
                

    def accuracy(self, activation):
        predictions = np.argmax(activation, axis=1)
        actuals = np.argmax(self.Y_train, axis=1)
        return np.mean(predictions == actuals)
    
    def test_accuracy(self, X_test, Y_test):
        Y_test = self.one_hot_encode(Y_test)
        activation = self.softmax(np.dot(X_test, self.Weights) + self.Bias)
        predictions = np.argmax(activation, axis=1)
        actuals = np.argmax(Y_test, axis=1)
        return np.mean(predictions == actuals)
    

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Test labels shape: {y_test.shape}")

x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

model = Logistic(x_train,y_train,10,0.001)
model.train_stochastic()
print("Test Accuracy: ",model.test_accuracy(x_test,y_test))

Training data shape: (50000, 32, 32, 3), Training labels shape: (50000, 1)
Test data shape: (10000, 32, 32, 3), Test labels shape: (10000, 1)
Epoch : 1 || Loss => 20.9535 || Accuracy => 1.0000
----------------------
Epoch : 2 || Loss => 20.9535 || Accuracy => 1.0000
----------------------
Epoch : 3 || Loss => 20.9535 || Accuracy => 1.0000
----------------------
Epoch : 4 || Loss => 20.9535 || Accuracy => 1.0000
----------------------
Epoch : 5 || Loss => 20.9535 || Accuracy => 1.0000
----------------------


KeyboardInterrupt: 

In [5]:
import numpy as np
from keras.datasets import mnist

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255
model = Logistic(train_images,train_labels,1000,0.001)
model.train_stochastic()
print("Test Accuracy: ",model.test_accuracy(test_images,test_labels))

model_batch = Logistic(train_images,train_labels,1000,0.001)
model_batch.train()
print("Test Accuracy: ",model_batch.test_accuracy(test_images,test_labels))

Epoch : 1 || Loss => 0.8611 || Accuracy => 0.8886
----------------------
Epoch : 2 || Loss => 0.8414 || Accuracy => 0.8977
----------------------
Epoch : 3 || Loss => 0.8329 || Accuracy => 0.9018
----------------------
Epoch : 4 || Loss => 0.8278 || Accuracy => 0.9041
----------------------
Epoch : 5 || Loss => 0.8243 || Accuracy => 0.9059
----------------------
Epoch : 6 || Loss => 0.8217 || Accuracy => 0.9068
----------------------
Epoch : 7 || Loss => 0.8196 || Accuracy => 0.9079
----------------------
Epoch : 8 || Loss => 0.8180 || Accuracy => 0.9083
----------------------
Epoch : 9 || Loss => 0.8166 || Accuracy => 0.9088
----------------------
Epoch : 10 || Loss => 0.8154 || Accuracy => 0.9093
----------------------
Epoch : 11 || Loss => 0.8144 || Accuracy => 0.9097
----------------------
Epoch : 12 || Loss => 0.8135 || Accuracy => 0.9103
----------------------
Epoch : 13 || Loss => 0.8127 || Accuracy => 0.9106
----------------------
Epoch : 14 || Loss => 0.8120 || Accuracy => 0.9

AttributeError: 'Logistic' object has no attribute 'one_hot_encode'